In [1]:
import sqlalchemy as db

In [2]:
# Path to database
db_file = "C:/users/jerry/Desktop/RestoredToEden.db"

# Set up SQLAlchemy connection
engine = db.create_engine('sqlite:///' + db_file)
cnxn = engine.connect()

metadata = db.MetaData()
metadata.reflect(cnxn)

InformationSheet = metadata.tables['InformationSheet']

KeyError: 'InformationSheet'

In [3]:
def get_table_contents(table):
    """ inputs: table containing items to query
        returns: dict of {col headings: col values}
    """
    query = db.select([table])
    headings = cnxn.execute(query).keys()
    query = db.select([table])
    values = cnxn.execute(query).fetchone()

    contents = {}
    # Create dict struct
    for i in range(len(headings)):
        contents[headings[i]] = values[i]

    return contents

get_table_contents(InformationSheet).values()

dict_values(['Simple instructions can be found on each product. [customer name], [detailed description inserted here from file]', 'It is recommended that you strip back your skincare routine to the basics and start with a blank slate. It is also recommended that you do not use other new skincare products while you are testing our products on your skin. The reason for this is you will not really know which of the products caused a reaction if you introduce too many at one time. If you usually do not have a problem with reactions on your skin, you should not have a reaction.', 'Store below 25 degrees celsius. Keep out of direct sunlight. Store in a dry place. Do not allow water to enter your product.', 'dwwd', 'We go to great lengths to hand select every ingredient. Please do not hesitate to let us know if you are not having the desired effect after using this product for the first 6 weeks. We will ensure that we analyse your needs further and make the necessary adjustments to your formu

In [4]:
def get_col_names(table):
    query = db.select([table])
    return cnxn.execute(query).keys()

get_col_names(InformationSheet)

['Recommendations For Use',
 'For Best Results',
 'Care & Storage Instructions',
 'Directions & Warnings Before Use',
 'Our promises to you',
 'Used By & Best Before Date']

In [5]:
def insert_query(table, data):
    """ data: dictionary of column name and its value
    """
    query = table.insert().values(data)
    cnxn.execute(query)
    
def update_query(table, data):
    query = db.update(table).values(data)
    cnxn.execute(query)
    

In [6]:
data = {"Recommendations For Use": "Simple instructions can be found on each product.  [customer name], [detailed description inserted here from file]",
        "For Best Results": "It is recommended that you strip back your skincare routine to the basics and start with a blank slate.  It is also recommended that you do not use other new skincare products while you are testing our products on your skin.  The reason for this is you will not really know which of the products caused a reaction if you introduce too many at one time.  If you usually do not have a problem with reactions on your skin, you should not have a reaction.",
        "Care & Storage Instructions": "Store below 25 degrees celsius.  Keep out of direct sunlight.  Store in a dry place.  Do not allow water to enter your product.",
        "Directions & Warnings Before Use": "Store below 25 degrees celsius.  Keep out of direct sunlight.  Store in a dry place.  Do not allow water to enter your product.",
        "Our promises to you": "We go to great lengths to hand select every ingredient.  Please do not hesitate to let us know if you are not having the desired effect after using this product for the first 6 weeks.  We will ensure that we analyse your needs further and make the necessary adjustments to your formulation.\nWe select only natural and organic ingredients in order to formulate products that are not only gentle on your skin but are also effective.  Each of our ingredients goes through a selection process to ensure the highest quality.\nPlease do not hesitate to contact us if you have any questions or concerns about any of our products.  We would be happy to help.",
        "Used By & Best Before Date": "This blend must be used within …. months of blending.\nDate Blended: [date blended]\nBest Before Date: [best before date]"
       }

update_query(InformationSheet, data)

In [7]:
query = db.select([InformationSheet])
paragraphs = cnxn.execute(query).fetchone()
col_names = get_col_names(InformationSheet)

paragraphs[-1]

'This blend must be used within …. months of blending.\nDate Blended: [date blended]\nBest Before Date: [best before date]'

In [72]:
import copy
# Delete column
metadata = db.MetaData()
metadata.reflect(cnxn)
InformationSheet = metadata.tables['InformationSheet']

# Get columns to keep
col_drop = ["new_column_name", "Field8"]
new_cols = []
for column in InformationSheet.columns:
    if not (column.name in col_drop):
        column_copy = copy.deepcopy(column)
        column_copy.table = None
        new_cols.append(column_copy)

# Save data to keep
new_content = get_table_contents(InformationSheet)
del new_content["Field8"]
del new_content["new_column_name"]

#new_table = db.Table("copy", metadata, *(column for column in new_cols))

# Section allows editing of information sheet
Store info in an excel sheet

In [2]:
from openpyxl import load_workbook
from datetime import *
import pandas as pd
import os

In [282]:
class ButtonWrapper:
    def __init__(self, button, label, text_box):
        self.df = None
        self.button = button
        self.label = label
        self.text_box = text_box
        
        self.button.clicked.connect(self.del_section)
        
    def del_section(self):
        df = pd.DataFrame({self.label.text(): [self.text_box.toPlainText()]})
        self.label.deleteLater()
        self.text_box.deleteLater()
        self.button.deleteLater()
        
        self.df = df

In [284]:
import sys
from PySide2.QtWidgets import *

paragraphs_df = pd.read_excel(info_sheet_path)

class Form(QDialog):
    def __init__(self, contents, parent=None):
        super(Form, self).__init__(parent)
        
        self.paragraphs_df = contents
        
        self.setWindowTitle("Information Sheet Editor")
        self.resize(700, 500)
        self.layout = QGridLayout()

        # Save button
        button = QPushButton("Save")
        button.clicked.connect(self.refresh)
        self.layout.addWidget(button, 0, 0)

        # Add new section button
        button = QPushButton("Add section")
        button.clicked.connect(self.add_section)
        self.layout.addWidget(button, 1, 0)
        
        self.del_buttons = []
        for i in range(contents.shape[1]):
            # Create label
            l = QLabel(list(contents)[i])
            self.layout.addWidget(l, 2*i+2, 0)
            
            # Create text boxes
            t = QTextEdit(contents.iloc[0,i])
            self.layout.addWidget(t, 2*i +2 + 1, 0)
            
            button = QPushButton("del")
            self.del_buttons.append(ButtonWrapper(button, l, t))
            self.layout.addWidget(button, 2*i +2 + 1, 1)

#         # Delete section button
#         button = QPushButton("Delete section")
#         button.clicked.connect(self.del_section)
#         self.layout.addWidget(button)

        # Set dialog layout
        self.setLayout(self.layout)

    def add_section(self):
        dialog = QDialog()
        dialog.setWindowTitle("New Section")
        layout = QVBoxLayout()

        # New section heading
        l = QLabel("Section Title")
        layout.addWidget(l)
        e = QLineEdit()
        layout.addWidget(e)

        # New section content
        l_content = QLabel("Section Content")
        layout.addWidget(l_content)
        e_content = QTextEdit()
        layout.addWidget(e_content)

        # Button adds new info to SQLite
        button = QPushButton("Ok!")
        button.clicked.connect(lambda: self.add_N_close(dialog, e.text(), e_content.toPlainText()))

        dialog.setLayout(layout)
        dialog.exec_() 

    def add_N_close(self, dialog, n_heading, n_body):
        new_section_df = pd.DataFrame({n_heading: [n_body]})
        self.paragraphs_df = pd.concat([paragraphs_df, new_section_df], axis=1)
        
        l = QLabel(n_heading)
        t = QTextEdit(n_body)
        self.text_boxes.append(t)
        
        # Insert new section before the buttons
        print(self.layout.count())
        pos = self.layout.rowCount()
        self.layout.addWidget(l, pos, 0)
        self.layout.addWidget(t, pos+1, 0)

        dialog.close()
    
     
    def refresh(self):
        for b_wrap in self.del_buttons:
            if b_wrap.df != None:
                print(b_wrap.df)
        # Export paragraph dfs to xlsx to load later
        #self.paragraphs_df.to_excel("test.xlsx")


In [280]:
# Read current info sheet
working_dir = os.getcwd()
info_sheet_path = working_dir + r"\BespokeAutoSystem\InformationSheet.xlsx"
paragraphs_df = pd.read_excel(info_sheet_path)
paragraphs_df

,Recommendations For Use,For Best Results,Care & Storage Instructions,Directions & Warnings Before Use,Our promises to you,Used By & Best Before Date
0,eek,Saradomin,Zaros,Zamorak,Bandos,Armadyl


# This Section opens excel worksheet template and writes to it!
1. Read input:
    - Ingredient list
    - Product name
    - Product type
    - Customer name
2. Open corresponding excel template to product type
3. Fill out header information
4. Go through ingredients and fill in main table entries
5. Run calc. function to fill grams based on product formulation
6. Output .xlsx

In [16]:
from openpyxl import load_workbook
from datetime import *
import pandas as pd
import os
import re

In [17]:
def ingredient_weight_real(ingredients, prod_type, ingredients_df):
    # Get template folder path
    path = os.getcwd() + "\\Formulation Templates\\"
    template_path = path + prod_type + " Worksheet.xlsx"
    # Load the excel sheet
    workbook = load_workbook(filename=template_path)
    sheet = workbook.active

    # Get ingredient % dict
    ww_dict, realloc_dict, phase_dict, assigned_vals, EOF = get_misc_items(sheet)
    print(assigned_vals)
    # Fill main table entries
    for i in range(len(ingredients)):
        ingredient_name = ingredients[i]

        # Get ingredient types ???????????/// Waiting for answer to simplify column
        type_str = str(ingredients_df.loc[ingredient_name]["TYPE OF INGREDIENT"]).lower()
        type_list = re.split("\s*[,]\s*", type_str)
        ingredient_type = type_list[-1]

        if "essential oil" in ingredient_type:
            ingredient_type = "eo " + ingredients_df.loc[ingredient_name]["ESSENTIAL OIL NOTE"].lower()
        
        if ingredient_type in ww_dict:
            if len(ww_dict[ingredient_type]) > 1:
                assigned_vals[ingredient_name] = ww_dict[ingredient_type].pop(0)
            else:
                assigned_vals[ingredient_name] = ww_dict[ingredient_type][0]

    print(assigned_vals)
    # Scale to 100
    tot = sum(assigned_vals.values())
    print(tot)
    if tot > 100:
        for key in assigned_vals.keys():
            assigned_vals[key] = assigned_vals[key] * 100/tot
    elif tot < 100:
        leftover = 100 - tot
        for key in assigned_vals.keys():
            assigned_vals[key] = assigned_vals[key] + leftover * assigned_vals[key]/tot
    else:
        pass
    return assigned_vals

In [18]:
def get_misc_items(sheet):
        ww_dict = {} 
        realloc_dict = {}
        phase_dict = {}
        assigned_vals = {}
        
        i = 7
        # Record the w/w% for all types
        while sheet[f"C{i}"].value != None:
            i_type = sheet[f"B{i}"].value.lower()
            if i_type not in ww_dict.keys():
                ww_dict[i_type] = [sheet[f"D{i}"].value]
            else:
                ww_dict[i_type].append(sheet[f"D{i}"].value)
            
            if ("this ingredient doesn't change" in sheet[f"B{i}"].value.lower()) or ("does not change" in sheet[f"B{i}"].value.lower()):
                assigned_vals[sheet[f"B{i}"].value] = sheet[f"D{i}"].value
            else:
                realloc_dict[f"D{i}"] = sheet[f"D{i}"].value
                
            phase_dict[i_type] = sheet[F"C{i}"].value
            i += 1
        EOF = i
        return ww_dict, realloc_dict, phase_dict, assigned_vals, EOF
    
def too_many_slots(realloc_dict, sheet):   
    # Write 0s to unallocated rows to make for delete
    for cell in realloc_dict.keys():
        sheet[cell] = 0 
        
    i = 7
    while sheet.cell(row=i, column=3).value != None:
        if sheet.cell(row=i, column=4).value == 0:
            sheet.delete_rows(i)
        else:
            i += 1
    return sheet

def too_few_slots(leftovers, phase_dict, EOF, sheet):

    for ingredient, weight in leftovers.items():
        if "ingredient doesn't change" in ingredient:
            continue
        # Get ingredient type and assign corresponding w/w%
        i_type = ingredients_df.loc[ingredient]["TYPE OF INGREDIENT"].lower().split(",")[-1] 
        # Insert at end of sheet
        sheet.insert_rows(EOF)
        # INCI name
        sheet[f"A{EOF}"] = ingredients_df.loc[ingredient]["INGREDIENT INCI NAME"]
        # name
        sheet[f"B{EOF}"] = ingredient  
        # Phase
        sheet[f"C{EOF}"] = phase_dict[i_type]
        # w/w%
        sheet[f"D{EOF}"] = weight
    return sheet

In [19]:
prod_type = "Day Cream"
ingredients = re.split("\s*[,]\s*", 
                       "Horse Chestnut Extract, Kaolin Clay, Aloe Vera Juice, Carrot Seed Essential Oil")

# prod_type = "Oil Cleanser"
# ingredients = ["Avocado", "Argan Oil", "Apricot Kernel Oil",
#               "Bergamot Essential Oil", "Cypress Essential Oil", "Frankincense Olibanum Essential Oil"]


customer_name = "Jane Doe"
# Load ingredients_df
ingredFname = "C:/Users/jerry/Desktop/Restored To Eden/Ingredients.csv"
ingredients_df = pd.read_csv(ingredFname)
ingredients_df.fillna("", inplace = True)
ingredients_df.set_index("INGREDIENT COMMON NAME", inplace=True)

# Get template folder path
path = os.getcwd() + "\\Formulation Templates\\"
template_path = path + prod_type + " Worksheet.xlsx"

In [97]:
# Load the excel sheet
workbook = load_workbook(filename=template_path)
sheet = workbook.active

ww_dict, realloc_dict, phase_dict, assigned_vals, EOF = get_misc_items(sheet)
# Get assigned values
assigned_vals = ingredient_weight_real(ingredients, prod_type, ingredients_df)
# Fill header info
sheet["B1"] = customer_name
sheet["B2"] = prod_type
sheet["B5"] = 100  # placeholder (varies)

print(assigned_vals)
i=0
# Fill main table entries
while len(assigned_vals) > 0:
    # End loop if all slots have been assigned
    if len(realloc_dict) <= 0:
        sheet = too_few_slots(assigned_vals, phase_dict, EOF, sheet)
        break
    
    ingredient_name = list(assigned_vals.keys())[i] 
    # Pass if static ingredient
    if "ingredient doesn't change" in ingredient_name.lower() or "does not change" in ingredient_name.lower():
        print(ingredient_name)
        j=7
         # Insert ingredient into a corresponding row
        while sheet[f"C{j}"].value != None:
            # Fill cell if ingredient is same type as template
            if ingredient_name.lower() == sheet[f"B{j}"].value.lower():
                # Fill w/w%
                sheet[f"D{j}"] = assigned_vals[ingredient_name]

                realloc_dict.pop(f"D{j}", None)
                assigned_vals.pop(ingredient_name)
                break 
            j+=1
    
        continue
    
 
    # Get ingredient types ???????????/// Waiting for answer to simply column
    type_list = str(ingredients_df.loc[ingredient_name]["TYPE OF INGREDIENT"]).lower()
    ingredient_type = re.split("\s*[,]\s*", type_list)[-1]

    if "essential oil" in ingredient_type:
        ingredient_type = "eo " + ingredients_df.loc[ingredient_name]["ESSENTIAL OIL NOTE"].lower()
    
    print(ingredient_type)
    print(ingredient_name)
    j = 7  # reset j
    # Insert ingredient into a corresponding row
    while sheet[f"C{j}"].value != None:
        # Fill cell if ingredient is same type as template
        if ingredient_type == sheet[f"B{j}"].value.lower():
            # Fill INCI
            sheet[f"A{j}"] = ingredients_df.loc[ingredient_name]["INGREDIENT INCI NAME"]
            # Fill ingredient name
            sheet[f"B{j}"] = ingredient_name
            # Fill w/w%
            sheet[f"D{j}"] = assigned_vals[ingredient_name]
            # Fill needs targted column if applicable
            try:
                if sheet[f"F{6}"].value.lower() == "needs targetting":
                    sheet[f"F{6}"] = ingredients_df.loc[ingredient_name]["SKIN PROBLEM"]
            except:
                print("No needs targetting column.")
                
            realloc_dict.pop(f"D{j}", None)
            assigned_vals.pop(ingredient_name)
            break 
        j+=1
        
    # if ingredient not assigned then +i
    if ingredient_name in assigned_vals.keys():
        i+=1

    
# Reallocate surplus %
if len(realloc_dict) > 0:
    sheet = too_many_slots(realloc_dict, sheet)

{'Humectant [this ingredient does not change]': 2.0, 'Emulsifier [this ingredient does not change]': 6.0, 'Preservative [this ingredient does not change]': 1.0, 'Antioxidant [this ingredient does not change]': 0.5}
{'Humectant [this ingredient does not change]': 2.0, 'Emulsifier [this ingredient does not change]': 6.0, 'Preservative [this ingredient does not change]': 1.0, 'Antioxidant [this ingredient does not change]': 0.5, 'Horse Chestnut Extract': 5.0, 'Aloe Vera Juice': 5.0, 'Carrot Seed Essential Oil': 0.6}
20.1
{'Humectant [this ingredient does not change]': 9.950248756218905, 'Emulsifier [this ingredient does not change]': 29.850746268656717, 'Preservative [this ingredient does not change]': 4.975124378109452, 'Antioxidant [this ingredient does not change]': 2.487562189054726, 'Horse Chestnut Extract': 24.87562189054726, 'Aloe Vera Juice': 24.87562189054726, 'Carrot Seed Essential Oil': 2.985074626865672}
Humectant [this ingredient does not change]
Emulsifier [this ingredient d

In [65]:
# Print spreadsheet:
k=7
while (k<24):
    print(k)
    print(sheet[f"B{k}"].value + ": " + str(sheet[f"D{k}"].value))
    k+=1

7
Anhydrous [this ingredient doesn't change]: 66.5
8
Avocado: 13.009540329575023
9
Argan Oil: 13.009540329575023
10
Bergamot Essential Oil: 0.4336513443191674
11
Cypress Essential Oil: 0.6938421509106678
12
Frankincense Olibanum Essential Oil: 0.4336513443191674
13
Antioxidant [this ingredient doesn't change]: 2
14
Apricot Kernel Oil: 13.009540329575023
15


TypeError: unsupported operand type(s) for +: 'NoneType' and 'str'

In [98]:
## Export
save_path = "C:/users/jerry/Desktop/Restored To Eden/test forms/"
workbook.save(save_path + customer_name + "-" + prod_type + ".xlsx")

In [26]:
# Load the excel sheet
workbook = load_workbook(filename=r"C:\Users\jerry\Desktop\Restored To Eden\Outputs\Jane Doe-Day Cream.xlsx")
sheet = workbook.active

In [29]:
i=7
while sheet[f"C{i}"].value != None:
    print(sheet[f"A{i}"].value)
    i+=1

None
Persea gratissima (Avocado) oil (certified organic)
Argania spinosa (Argan) oil (certified organic)
Prunus armeniaca (Apricot) kernel oil (certified organic)
None
None
Citrus Bergamo
Cupressus sempervirens (Cypress) essential oil
Boswellia carterii (Frankincense) essential oil
None


In [34]:
from __future__ import print_function
import pickle
import os.path
from googleapiclient.discovery import *
from google_auth_oauthlib.flow import *
from google.auth.transport.requests import *
import io
from googleapiclient.http import *
import pandas as pd

# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/drive',
          'https://www.googleapis.com/auth/spreadsheets']

"""Shows basic usage of the Drive v3 API.
Prints the names and ids of the first 10 files the user has access to.
"""
creds = None
# The file token.pickle stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token.pickle'):
    with open('token.pickle', 'rb') as token:
        creds = pickle.load(token)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.pickle', 'wb') as token:
        pickle.dump(creds, token)

service = build('drive', 'v3', credentials=creds)

In [35]:
def gdrive_get_file(filename):
    results = service.files().list(q=f"name contains '{filename}'",
                    spaces='drive',
                    fields='nextPageToken, files(id, name, mimeType)').execute()

    file_id = results.get('files', [])[0]['id']
    file_type = results.get('files', [])[0]['mimeType']
    
    # Check file type and send request appropriately
    if 'application/vnd.google-apps' in file_type:
        request = service.files().export_media(fileId=file_id, 
           mimeType="application/vnd.openxmlformats-officedocument.spreadsheetml.sheet")
    else:
        print(file_id)
        request = service.files().get_media(fileId=file_id)
        
    fh = io.BytesIO()
    downloader = MediaIoBaseDownload(fh, request)

    done = False
    while done is False:
        status, done = downloader.next_chunk()
       # print("Download %d%%." % int(status.progress() * 100))

    return fh, file_id

In [36]:
# Push file to gdrive
def push_file(self, file_id, filename, path, **kwargs):
    """ Upload to google drive, update existing file
    """
    file_metadata = {"name": filename}

    media = MediaFileUpload(path, 
        mimetype="application/vnd.openxmlformats-officedocument.spreadsheetml.sheet")
    
    if kwargs.get("action") == "update":
        f = self.service.files().update(fileId=file_id, 
                                        body=file_metadata,
                                        media_body=media,
                                        fields="id").execute()
    else:
        f = self.service.files().create(body=file_metadata,
                                        media_body=media,
                                        fields="id").execute()

In [3]:
import pandas as pd

ingredients_df = pd.read_csv("C:/Users/jerry/Desktop/Restored To Eden/RTE Ingredients Database.csv")
ingredients_df.set_index("INGREDIENT COMMON NAME", inplace=True)
ingredients_df

,INGREDIENT INCI NAME,In Stock,Quantity in Stock (in grams),Supplier Name,CAS NO.,NICNAS Check Completed,Product Type,Skin Type,TYPE OF INGREDIENT,ESSENTIAL OIL NOTE,...,VISCOSITY,ABSORPTION RATE,COMEDOGENIC RATING,Composition,Recommended Concentration,Allergens,Relevant References,Other Important Information,4,K
INGREDIENT COMMON NAME,,,,,,,,,,,,,,,,,,,,,
Acai Oil Certified Organic,Euterpe oleracea (Acai) oil (certified organic),Yes,NaN,AWO,NaN,No,Anhydrous,"Dry Skin,Sensitive & Eczema Prone Skin,Mature ...","Carrier Oil,anhydrous base",NaN,...,Light,Fast,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aloe Vera Juice,Aloe Barbadensis (Aloe Vera) juice,Yes,NaN,ND,NaN,Yes,NaN,Mature & Aging Skin,"Extract,aqueous high performance",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Aloe Vera powder and purified Aqua - and prese...,NaN,NaN,NaN
Alpinia Leaf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mature & Aging Skin,"Extract,aqueous high performance",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Alpinia Root,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mature & Aging Skin,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Apricot Kernel Oil,Prunus armeniaca (Apricot) kernel oil (certifi...,Yes,NaN,AWO,NaN,Yes,"Anhydrous,Emulsion","Dry Skin,Sensitive & Eczema Prone Skin,Mature ...","Carrier Oil,anhydrous base",NaN,...,Light,Fast,2.0,NaN,NaN,NaN,Review Article\nPolyphenols as active ingredie...,Thinner consistency\nCleansing properties\n\nA...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Vitamin E Natural (Mixed tocopherols 70%),Mixed tocopherols 70% (Vitamin E natural),Yes,NaN,AWO,7695-91-2,No,"Anhydrous,Emulsion",Mature & Aging Skin,"Antioxidant,anhydrous high performance",NaN,...,NaN,NaN,NaN,NaN,0.5%-2%,NaN,NaN,"Components: Mixed Tocopherols (d-alpha-, d-bet...",NaN,NaN
Walnut Oil,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,"Carrier Oil,anhydrous base",NaN,...,NaN,Medium,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
White Willow Bark,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sensitive & Eczema Prone Skin,"Extract,aqueous high performance",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Config File Testing

In [4]:
import json
# Json is a dictionary in python
Config = {"ConstraintSettings": {
    "DayCream": {"Viscosity": "Light", "Absorbency": "Fast"},
    "ManCream": {"Viscosity": "Medium", "Absorbency": "Medium"},
    "NightCream": {"Viscosity": "Heavy", "Absorbency": "Slow"},
    "OilCleanser": {"Viscosity": "Medium", "Absorbency": "Medium"}
},
          "Viscosity": ["light", "medium", "heavy"],
          "Absorbency": ["slow", "medium", "fast"],
          "ComedogenicRating": [0, 1, 2 ,3 ,4 ,5]}
# Writing config file to output
json_obj = json.dumps(Config, indent=4)
with open("config.json", "w") as outfile:
    outfile.write(json_obj)
    
# Reading config file
with open("config.json") as json_file:
    data = json.load(json_file)
print(data)

{'ConstraintSettings': {'DayCream': {'Viscosity': 'Light', 'Absorbency': 'Fast'}, 'ManCream': {'Viscosity': 'Medium', 'Absorbency': 'Medium'}, 'NightCream': {'Viscosity': 'Heavy', 'Absorbency': 'Slow'}, 'OilCleanser': {'Viscosity': 'Medium', 'Absorbency': 'Medium'}}, 'Viscosity': ['light', 'medium', 'heavy'], 'Absorbency': ['slow', 'medium', 'fast'], 'ComedogenicRating': [0, 1, 2, 3, 4, 5]}


# Find distance between 2 lines

In [25]:
# Get coordinates by coverting categorical data to numeric
# Line coords: (x=Viscosity, y=Absorbency, z=Comedogenic rating)
product_type = "OilCleanser"
# Find coordinates of points (a) for optimum
try:
    x1_str = data["ConstraintSettings"][product_type]["Viscosity"].lower()
    x1 = [option.lower() for option in data["Viscosity"]].index(x1_str)
except:
    x1 = 0

try:
    y1_str = data["ConstraintSettings"][product_type]["Absorbency"].lower()
    y1 = [option.lower() for option in data["Absorbency"]].index(y1_str)
except:
    y1 = 0

try:
    z1_str = data["ConstraintSettings"][product_type]["ComedogenicRating"].lower()
    z1 = [option.lower() for option in data["ComedogenicRating"]].index(z1_str)
except:
    z1 = 0
    
a1 = (x1, y1, z1)

# a2 is the point of the selected ingredient list - take average of all ingredients?
# Since all points go through origin the directional vector is just the vector itself

(1, 1, 0)

In [36]:
import numpy as np
# distance between 2 skew lines
# Points
a1 = [0, 2, -1]
a2 = [1, 0, -1]
# Direction vectors
b1 = [1, 1, 2]
b2 = [1, 1, 3]

PQ = []
for i in range(len(a1)):
    PQ.append(a2[i] - a1[i])
    
np.linalg.norm(np.dot(PQ, np.cross(b1, b2))) / np.linalg.norm(np.cross(b2, b1))

2.1213203435596424

In [19]:
class a:
    val = 10
class b:
    def __init__(self, a):
        self.a = a
    
aob = a()
bob = b(aob)

In [25]:
bob.a.val = 10
print(bob.a.val)
print(aob.val)

10
10


In [5]:
import re

re.split("\s*[,]\s*", "carrier oil,      aqueous")

['carrier oil', 'aqueous']

In [7]:
import pandas as pd

ingredFname = "C:/Users/jerry/Desktop/Restored To Eden/Ingredients.csv"
ingredients_df = pd.read_csv(ingredFname)
ingredients_df.fillna("", inplace = True)
ingredients_df = ingredients_df.applymap(lambda x:str(x).upper())

In [11]:
ingredients_df

,INGREDIENT COMMON NAME,INGREDIENT INCI NAME,In Stock,Quantity in Stock (in grams),Supplier Name,CAS NO.,NICNAS Check Completed,Product Type,Skin Type,TYPE OF INGREDIENT,...,VISCOSITY,ABSORPTION RATE,COMEDOGENIC RATING,Composition,Recommended Concentration,Allergens,Relevant References,Other Important Information,4,K
0,ACAI OIL CERTIFIED ORGANIC,EUTERPE OLERACEA (ACAI) OIL (CERTIFIED ORGANIC),YES,,AWO,,NO,ANHYDROUS,"DRY SKIN,SENSITIVE & ECZEMA PRONE SKIN,MATURE ...","CARRIER OIL,ANHYDROUS BASE",...,LIGHT,FAST,2.0,,,,,,,
1,ALOE VERA JUICE,ALOE BARBADENSIS (ALOE VERA) JUICE,YES,,ND,,YES,,MATURE & AGING SKIN,"EXTRACT,AQUEOUS HIGH PERFORMANCE",...,,,,,,,ALOE VERA POWDER AND PURIFIED AQUA - AND PRESE...,,,
2,ALPINIA LEAF,,,,,,,,MATURE & AGING SKIN,"EXTRACT,AQUEOUS HIGH PERFORMANCE",...,,,,,,,,,,
3,ALPINIA ROOT,,,,,,,,MATURE & AGING SKIN,,...,,,,,,,,,,
4,APRICOT KERNEL OIL,PRUNUS ARMENIACA (APRICOT) KERNEL OIL (CERTIFI...,YES,,AWO,,YES,"ANHYDROUS,EMULSION","DRY SKIN,SENSITIVE & ECZEMA PRONE SKIN,MATURE ...","CARRIER OIL,ANHYDROUS BASE",...,LIGHT,FAST,2.0,,,,REVIEW ARTICLE\nPOLYPHENOLS AS ACTIVE INGREDIE...,THINNER CONSISTENCY\nCLEANSING PROPERTIES\n\nA...,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,VITAMIN E NATURAL (MIXED TOCOPHEROLS 70%),MIXED TOCOPHEROLS 70% (VITAMIN E NATURAL),YES,,AWO,7695-91-2,NO,"ANHYDROUS,EMULSION",MATURE & AGING SKIN,"ANTIOXIDANT,ANHYDROUS HIGH PERFORMANCE",...,,,,,0.5%-2%,,,"COMPONENTS: MIXED TOCOPHEROLS (D-ALPHA-, D-BET...",,
106,WALNUT OIL,,NO,,,,,,,"CARRIER OIL,ANHYDROUS BASE",...,,MEDIUM,2.0,,,,,,,
107,WHITE WILLOW BARK,,,,,,,,SENSITIVE & ECZEMA PRONE SKIN,"EXTRACT,AQUEOUS HIGH PERFORMANCE",...,,,,,,,,,,
108,WITCH HAZEL INFUSED WATER,HAMAMELIS VIRGINIANA,YES,,,,YES,,SENSITIVE & ECZEMA PRONE SKIN,AQUEOUS HIGH PERFORMANCE,...,,,,,,,,,,


In [3]:
import docx
f = open("Product Information/Day Cream Instructions.docx", 'rb')
fullText = []
doc = docx.Document(f)
for para in doc.paragraphs:
    fullText.append(para.text)

fullText[-1]

'…, use in the morning after cleaning your face and cleansing.  I would recommend using a small amount and dotting it over your face then massaging this into your skin.  Only apply a very thin layer to the areas you feel are oilier (e.g. T-Zone).  I have used oils that will balance sebum production and oils that have a lower comedogenic rating so your pores do not get clogged.'

In [32]:
from pathlib import Path
import os
from os import listdir
from os.path import isfile, join

path = Path(os.getcwd())
print(path)
print(path.parent.parent)

mypath = str(Path(os.getcwd())) + "\\Outputs"
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
onlyfiles

C:\Users\jerry\Desktop\Restored To Eden
C:\Users\jerry


['Jane Doe-Day Cream.xlsx']

In [15]:
os.getcwd() + r"/infosheetAss"

'C:\\Users\\jerry\\Desktop\\Restored To Eden/infosheetAss'

In [56]:
info = pd.read_excel("C:/Users/jerry/Desktop/Restored To Eden/InformationSheet.xlsx")
new = "efewfwewfefew"
try:
    info.insert(info.shape[1]-1, "Ingredients", new)
except:
    pass
info

,Care & Storage Instructions,Directions & Warnings Before Use,Our promises to you,new section,Ingredients,dragonstone
0,Zaros,Zamorak,Bandos,stormborn,efewfwewfefew,Daenarys
